In [2]:
import sys
sys.path.append("/home/cyprien/Documents/github/pytorch-forecasting")

import hashlib

from data_factory.preprocessing import *
from utilities.config import load_config

from pytorch_forecasting import TimeSeriesDataSet

logging.basicConfig(level=DEBUG)
logger = logging.getLogger(__name__)
logger.setLevel(DEBUG)

In [3]:
config_file = '../config/config.yml'
use_previous_files = False
export = False

In [4]:


config = load_config(config_file)

model = config['model']
model_config = config[model]

# Create variables from config
#  data loading
save_folder = config['data']['save']
train_file = config['data']['train_path'] + config['data']['stock_prices']
test_file = config['data']['test_path'] + config['data']['stock_prices']
#  TimeSeries settings
max_prediction_length = model_config['sliding_window']['max_prediction_length']
min_prediction_length = model_config['sliding_window']['min_prediction_length']
max_encoder_length = model_config['sliding_window']['max_encoder_length']
min_encoder_length = model_config['sliding_window']['min_encoder_length']
batch_size = model_config['sliding_window']['batch_size']

related_stocks = model_config['related_stock']
train_val_split = model_config['train_val_split']
scale = model_config['manual_scale']

# define file name for saving StockPricesLoader with specific config
hash_ = hashlib.md5(model_config.__str__().encode('utf-8')).hexdigest()
export_file_name = f"{save_folder}/export_{hash_}.p"

logger.debug(f'Export file {export_file_name}')
logger.debug(f'Use config {config}')


DEBUG:__main__:Export file ../data/save//export_c812f38e2002410441e38cf0c23469e2.p
DEBUG:__main__:Use config {'device': 'cpu', 'seed': False, 'model': 'temporal_fusion_transformer', 'data': {'save': '../data/save/', 'suppl': '../data/suppl/', 'train_path': '../data/jpx-tokyo-stock-exchange-prediction/train_files/', 'test_path': '../data/jpx-tokyo-stock-exchange-prediction/supplemental_files/', 'financials': 'financials.csv', 'stock_prices': 'stock_prices.csv', 'options': 'options.csv', 'secondary_stock_price': 'secondary_stock_price.csv', 'trades': 'trades.csv', 'cosine': 'cosine_df.csv'}, 'rnn': {'sliding_window': {'max_prediction_length': 10, 'min_prediction_length': 10, 'max_encoder_length': 150, 'min_encoder_length': 150, 'batch_size': 64}, 'train_val_split': 1, 'related_stock': 2, 'manual_scale': True, 'hidden_size': 20, 'layers': 3, 'dropout': 0}, 'temporal_fusion_transformer': {'sliding_window': {'max_prediction_length': 10, 'min_prediction_length': 10, 'max_encoder_length': 150

### Load

In [5]:
from data_factory.prepared_data import PreparedData

data: PreparedData = PreparedData.from_file("../data/save/augmented_data.pkl")
df_train = data.train
df_val = data.val
df_test = data.test

### TimeSeries

#### Train

In [ ]:
additional_cols = [f'Close_scaled_top_{i}' for i in range(related_stocks)]
logger.info(f'Add columns {additional_cols}')
time_varying_unknown_reals = ['Close_scaled', 'Open_scaled', 'High_scaled', 'Low_scaled', 'Volume_scaled', 'AdjustmentFactor', 'ExpectedDividend'] + additional_cols

In [ ]:
df_train_timeseries = TimeSeriesDataSet(
    df_train,
    time_idx='Timestamp',
    target='Close_scaled',
    group_ids=['SecuritiesCode'],
    allow_missing_timesteps=False,

    static_categoricals=[],
    time_varying_unknown_reals=time_varying_unknown_reals,
    time_varying_unknown_categoricals=['SupervisionFlag'],
    time_varying_known_reals=[],

    min_encoder_length=min_encoder_length,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_prediction_length=min_prediction_length,

    scalers={col: None for col in list(set(time_varying_unknown_reals) - {'Close_scaled'})},
    target_normalizer=None,
    add_relative_time_idx=model == 'temporal_fusion_transformer',
    add_target_scales=False,
    add_encoder_length=False,
)

#### Validation

In [ ]:
df_val_timeseries = TimeSeriesDataSet.from_dataset(
    df_train_timeseries, df_val,
    predict=False,
    stop_randomization=True,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length
)

#### Test

In [ ]:
# Test timeseries
df_test_timeseries = TimeSeriesDataSet.from_dataset(
    df_train_timeseries, df_test,
    allow_missing_timesteps=False,
    predict=False,
    stop_randomization=True,
    min_prediction_idx=df_test[df_test.is_testing == True].Timestamp.min() + 1,
    min_prediction_length=max_prediction_length,
    max_prediction_length=max_prediction_length
)
logger.debug('test timeseries created')
logger.info('Timeseries created')


In [ ]:
## Batch size of test set to predict one SecuritiesCode at a time.
# Created to make debug easier.

test_set_size = df_test[df_test.is_testing == True].Timestamp.max() - df_test[df_test.is_testing == True].Timestamp.min() + 1

### DataLoaders
*Just to make sure it works, not exported*

In [ ]:
train_dl = df_train_timeseries.to_dataloader(train=True, batch_size=batch_size, num_workers=12)

val_dl = df_val_timeseries.to_dataloader(train=False, batch_size=batch_size, num_workers=12, shuffle=False)

test_dl = df_test_timeseries.to_dataloader(
    batch_size=test_set_size,
    num_workers=12,
    shuffle=False
)

### Export

In [ ]:
from data_factory.prepared_data import TimeSeriesData

data = TimeSeriesData(df_train_timeseries, df_val_timeseries, df_test_timeseries, test_set_size)

In [ ]:
data.export('../data/save/timeseries_data.pkl')

#### Experiments

In [ ]:
for X, (y, _) in df_train_timeseries:
    print(y.shape)
    print(X.keys())

    print(X['x_cont'].shape)

    break